# Exploratory Data Analysis and Cleaning

## Date: OCT 10, 2023

## Introduction
This notebook cleans the data for the lending club accepted loans. However, there are some rows that have values shifted column to column, resulting in different data types within a single column.  

### Table-of-contents


1. [Introduction](#Introduction)
   - [Table-of-contents](#Table-of-contents)
   - [Data Dictionary](#Data-Dictionary)
   - [Import-Librarys](#Import-Librarys)
   - [Define-Functions](#Define-Functions)
   - [Handle-mixed-data-types](#Handle-mixed-data-types)
3. [Data Cleaning](#Data-Cleaning)
   - [Initial Exploration](#Initial-Exploration)
   - [Explore Columns to drop](#Explore-Columns-to-drop)
3. [Exploratory Data Analysis](#Exploratory-Data-Analysi6)
4. [Modeling](#Modeli7g)
5. [Conclusion](#Conclusion)


## Data-Dictionary

- how much data is lost per column due to cleaning
- no data truncation subprocess, load from drive
- explain why data is missing etc
- use print instead of markdown so it updates for real data
- table of contents
- dummy variables
- human readable then machine readable
- optimize after
- write a func to clean columns?
- column pruning after fixing problem row
- reduce memory size
- optimize column size later
- follow PEP8 style guide
- #### NO NAME

#### Import-Librarys

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

#### Define-Functions

When initially loading in the dataset, Pandas raised a DtypeWarning over mixed datatypes within various columns. Setting low_memory = False while breaking the CSV into chunks allows Pandas to load an entire chunk before guessing the data types. The mixed_data_types function is stilled called as a sanity check.


In [2]:
def mixed_data_types(df:pd.DataFrame) -> bool:
    '''
    Takes in a dataframe and checks for columns with mixed data types
    If none are found return False, else True
    
    :param df: The dataframe to be checked
    :type df: obj
    :return bool: True if found, false if none were found
    :type return: bool
    '''
    
    #loop through each column
    for column in df:

        #filter outint datatypes coming from Nan and get unique data types
        unique_types = df[column].dropna(inplace=False).apply(type).unique()

        #if there are more than 1 datatype in a column
        if unique_types.size > 1:
            return True
    return False

In [3]:
def df_cleaner(remove_cols:list, df:pd.DataFrame) -> pd.DataFrame:
    '''
    Takes in a dataframe and removes /  optimizes

    :param df: The dataframe to be checked
    :type df: Pandas df
    :return df: Returns the cleaned dataframe
    :type return: bool

    '''    

#### Handle-mixed-data-types

Due to the size of the dataset, it is read in chunks. After each chunk is read and checked for mixed data types, it is randomly sampled and then placed within a list. The different samples are then combined into a single sample representative of the whole dataset. EDA will be performed on this single sample.

In [4]:
chunk_size = 5*100000
sample_size =  100000
random_state = 11

assert sample_size < chunk_size, f"Cannot take a sample of {sample_size} rows out of {chunk_size} rows"

print(f'Chunk size: {chunk_size} rows')
print(f'Rows to be sampled: {sample_size} rows')


sampled_dataframes = []
try:
    data_destination = Path('../Data/Lending_club/accepted_2007_to_2018Q4.csv')

    #split the csv into chunks and iterate over each chunk
    with pd.read_csv(data_destination, chunksize=chunk_size, low_memory = False) as reader:
        for count,chunk in enumerate(reader):
            
            if mixed_data_types(df=chunk) == True:
                raise Exception("Mixed data types found")
            
            sampled_df = chunk.sample(n=sample_size, random_state=random_state)
            sampled_dataframes.append(sampled_df)
            print(f"{count} sampled dataframe shape: {sampled_df.shape}")
        print('Finished')

except FileNotFoundError as e:
    print(e.args[1])
    print('Check file name and location')
    
except Exception as e:
    print(e.args[1])

Chunk size: 500000 rows
Rows to be sampled: 100000 rows
0 sampled dataframe shape: (100000, 151)
1 sampled dataframe shape: (100000, 151)
2 sampled dataframe shape: (100000, 151)
3 sampled dataframe shape: (100000, 151)
4 sampled dataframe shape: (100000, 151)
Finished


There are no duplicate datatypes within any columns. The random samples can be combined into a single sample dataframe. This sample will be used as the dataset to use.

In [5]:
sample_accepted_df = pd.concat(sampled_dataframes, ignore_index=False)

&nbsp;

## Data Cleaning

In [6]:
pd.set_option('display.max_columns', None)

Set pandas display option to show all columns

&nbsp;

### Initial Exploration

***Display the first 5 rows*** 

In [7]:
sample_accepted_df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
408717,38538355,NaN,11550.0,11550.0,11550.0,36 months,11.99,383.58,B,B5,Sales Associate,10+ years,RENT,33754.95,Source Verified,Jan-2015,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,900xx,CA,26.21,0.0,Dec-1991,675.0,679.0,0.0,NaN,72.0,9.0,1.0,7238.0,49.2,20.0,f,0.00,0.00,5158.650000,5158.65,2805.61,1014.80,0.0,1338.24,240.8832,Nov-2015,383.58,NaN,Oct-2016,554.0,550.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,20413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14700.0,NaN,NaN,NaN,3.0,2268.0,1630.0,81.3,0.0,0.0,150.0,277.0,20.0,6.0,1.0,39.0,NaN,NaN,NaN,0.0,3.0,4.0,3.0,6.0,4.0,7.0,15.0,4.0,9.0,0.0,0.0,0.0,1.0,100.0,66.7,1.0,0.0,32700.0,20413.0,8700.0,18000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
229511,54313184,NaN,16775.0,16775.0,16775.0,60 months,15.61,404.47,D,D1,Teller,10+ years,MORTGAGE,38168.00,Source Verified,Jul-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,672xx,KS,14.40,0.0,Jul-2004,660.0,664.0,0.0,NaN,NaN,26.0,0.0,17032.0,53.8,41.0,w,5808.34,5808.34,17782.130000,17782.13,10966.66,6815.47,0.0,0.00,0.0000,Mar-2019,404.47,Apr-2019,Mar-2019,744.0,740.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,214.0,68813.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31674.0,NaN,NaN,NaN,6.0,3277.0,3681.0,82.1,0.0,0.0,122.0,131.0,10.0,10.0,2.0,22.0,NaN,12.0,NaN,0.0,6.0,7.0,8.0,14.0,7.0,24.0,32.0,7.0,26.0,0.0,0.0,0.0,1.0,100.0,75.0,0.0,0.0,91524.0,1

&nbsp;

***Dataframe shape***

In [8]:
rows, columns = sample_accepted_df.shape
print(f'Dataframe rows: {rows}')
print(f'Dataframe columns: {columns}')

Dataframe rows: 500000
Dataframe columns: 151


&nbsp;

***Dataframe info***

In [9]:
sample_accepted_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 408717 to 2006217
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 579.8+ MB


Of the 151 columns, 113 are float64 and 38 are objects. The dataframe takes up approximatly 580 MB.
Note:
- The numeric columns are all float64 and the object columns. These columns can be optimized later to save memory space and decrease computation time by changing the datatypes.
- There is no datetime column

&nbsp;

***Describe Dataframe***

In [10]:
sample_accepted_df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,499992.000000,499992.000000,499992.000000,499992.000000,499992.000000,4.999910e+05,499612.000000,499988.000000,499992.000000,499992.000000,499988.000000,244645.000000,80269.000000,499988.000000,499988.000000,4.999920e+05,499562.000000,499988.000000,499992.000000,499992.000000,499992.000000,499992.000000,499992.000000,499992.000000,4.999920e+05,499992.000000,499992.000000,499992.000000,499992.000000,499992.000000,499969.000000,129402.000000,499992.0,2.792400e+04,27923.000000,499988.000000,485935.000000,4.859350e+05,327517.000000,327517.000000,327517.000000,327517.000000,317069.000000,3.275170e+05,280016.000000,327517.000000,327517.000000,327517.000000,327462.000000,4.859350e+05,327517.000000,327517.000000,327517.000000,490018.000000,485924.000000,484319.000000,484074.000000,499969.000000,499988.000000,470351.000000,485935.000000,485935.000000,485935.000000,490018.000000,484669.000000,115213.000000,435627.000000,164061.000000,485935.000000,485935.000000,485935.000000,488282.000000,485935.000000,485935.000000,485935.000000,485935.000000,485935.000000,488282.000000,466483.000000,485935.000000,485935.000000,485935.000000,485909.000000,484248.000000,499714.000000,499974.000000,4.859350e+05,4.900180e+05,490018.000000,4.859350e+05,24603.000000,24604.000000,24604.000000,24604.000000,24604.000000,24604.000000,24179.000000,24604.000000,24604.000000,24604.000000,24604.000000,8150.000000,2542.0,2542.000000,2542.0,2542.000000,2021.000000,2542.000000,2542.000000,7398.000000,7398.000000,7398.000000
mean,NaN,15014.911589,15010.218413,14993.045814,13.105586,445.405616,7.807225e+04,18.833484,0.307261,698.732270,702.732492,0.571258,34.555871,72.126462,11.600126,0.200049,1.664924e+04,50.044462,24.055763,4264.585065,4263.611967,11983.934957,11966.827390,9427.047136,2414.427206,1.527768e+00,140.932849,23.577253,3430.033461,687.707851,675.593820,0.018635,44.157648,1.0,1.226873e+05,19.265859,0.004156,224.714733,1.424189e+05,0.936855,2.780295,0.677171,1.571070,21.148062,3.551788e+04,69.133899,1.290886,2.753457,5780.577222,57.049670,3.466175e+04,1.009401,1.475887,2.034826,4.532252,13539.739056,11478.07804

Some key points:

- Loan Amount
  
    - Average Loan Amount is ~ 15,000 USD with a standard deviation of 9240 USD, having a max of 40,000 USD and minimum of 500 USD. This follows LendingClubs  policies for minimum and maximum loan amounts.

- Funded amount
    - Nearly identical to the loan amount

- Funded amount by investors
    - Very similar to the  

- Interest Rate
    - The interest rates are quite high. An average of 13%, with a minimum of 5.3% and a maximum of 31%.


   

***Dataframe Null Values***

In [11]:
sample_accepted_df.isnull().sum()

id                            0
member_id                500000
loan_amnt                     8
funded_amnt                   8
funded_amnt_inv               8
                          ...  
settlement_status        492602
settlement_date          492602
settlement_amount        492602
settlement_percentage    492602
settlement_term          492602
Length: 151, dtype: int64

We can create a list to keep track of the columns to drop.

In [12]:
columns_to_drop = []

We can drop columns that are linked to LendingClubs internal tracking of the loans. 

In [13]:
columns_to_drop.extend(['member_id','url'])
print(columns_to_drop)

['member_id', 'url']


### Explore Columns to drop

***Hardship Loans***

In [15]:
#fetch the value counts for the for the hardships flags
hardships = sample_accepted_df['hardship_flag'].value_counts()

#pull out the count of hardships loans, as well as the total number of rows
yes_hardship = hardships.iloc[1]
df_row_count = sample_accepted_df.shape[0]

display(hardships)

hardship_str = f"""\nSince there are {yes_hardship} hardship loans,
representing only {(yes_hardship/df_row_count)*100}% of the dataframe,
we can safely drop the associated rows and columns. Note however, 
our analysis now cannot be applied to hardship loans."""

print(hardship_str)

hardship_flag
N    499787
Y       205
Name: count, dtype: int64


Since there are 205 hardship loans,
representing only 0.041% of the dataframe,
we can safely drop the associated rows and columns. Note however, 
our analysis now cannot be applied to hardship loans.


***Drop Hardship loans***

In [16]:
rows_to_remove = sample_accepted_df.loc[sample_accepted_df['hardship_flag'] == 'Y'].index
sample_accepted_df.drop(rows_to_remove, inplace=True)

Check rows have been dropped

In [17]:
sample_accepted_df['hardship_flag'].value_counts()

hardship_flag
N    499787
Name: count, dtype: int64

The hardship loans have been dropped. We can now add the associated columns to the drop column list

In [19]:
columns_to_drop.extend(['hardship_flag', 'hardship_type',
                        'hardship_reason', 'hardship_status',
                        'hardship_amount', 'hardship_start_date',
                        'hardship_end_date', 'deferral_term ',
                        'hardship_length', 'hardship_dpd',
                        'hardship_loan_status', 'payment_plan_start_date',
                        'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
                        'hardship_last_payment_amount'])

***Drop funded_amnt***

In [47]:
print(len(sample_accepted_df['emp_title'].unique()))
sample_accepted_df['loan_status'].value_counts()

9975


loan_status
Fully Paid            14154
Charged Off            3571
Current                2120
Late (31-120 days)      106
In Grace Period          41
Late (16-30 days)         8
Name: count, dtype: int64

In [48]:
sample_accepted_df['emp_title'].value_counts()

emp_title
Teacher                                     421
Manager                                     330
Owner                                       170
Supervisor                                  158
RN                                          156
                                           ... 
Team Lead -Wells Fargo Merchant Services      1
Accounting Dept                               1
Principle Network Engineer                    1
Senior Staff Assistant                        1
Business Banking Specialist                   1
Name: count, Length: 9974, dtype: int64

In [39]:
colms_to_drop = ['url']
nulls = sample_accepted_df.isnull().sum()
for index, value in nulls.items():
    print(index, value)

id 0
member_id 20000
loan_amnt 0
funded_amnt 0
funded_amnt_inv 0
term 0
int_rate 0
installment 0
grade 0
sub_grade 0
emp_title 1212
emp_length 1202
home_ownership 0
annual_inc 0
verification_status 0
issue_d 0
loan_status 0
pymnt_plan 0
url 0
desc 19999
purpose 0
title 132
zip_code 0
addr_state 0
dti 1
delinq_2yrs 0
earliest_cr_line 0
fico_range_low 0
fico_range_high 0
inq_last_6mths 0
mths_since_last_delinq 9662
mths_since_last_record 16381
open_acc 0
pub_rec 0
revol_bal 0
revol_util 11
total_acc 0
initial_list_status 0
out_prncp 0
out_prncp_inv 0
total_pymnt 0
total_pymnt_inv 0
total_rec_prncp 0
total_rec_int 0
total_rec_late_fee 0
recoveries 0
collection_recovery_fee 0
last_pymnt_d 15
last_pymnt_amnt 0
next_pymnt_d 17725
last_credit_pull_d 0
last_fico_range_high 0
last_fico_range_low 0
collections_12_mths_ex_med 0
mths_since_last_major_derog 14247
policy_code 0
application_type 0
annual_inc_joint 19851
dti_joint 19851
verification_status_joint 19851
acc_now_delinq 0
tot_coll_amt 0
t

***Handle Null values***

In [35]:

for column in sample_accepted_df.columns:
    row_null_count = sample_accepted_df[column].isnull().sum()
    row_count = sample_accepted_df[column].shape[0]
    
    print(round((row_null_count / row_count)*100,2), '%')
    #print(sample_accepted_df[column])

0.0 %
100.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
6.06 %
6.01 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
100.0 %
0.0 %
0.66 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
48.31 %
81.9 %
0.0 %
0.0 %
0.0 %
0.06 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.08 %
0.0 %
88.62 %
0.0 %
0.0 %
0.0 %
0.0 %
71.24 %
0.0 %
0.0 %
99.26 %
99.26 %
99.26 %
0.0 %
0.0 %
0.0 %
2.09 %
2.09 %
2.09 %
2.09 %
4.61 %
2.09 %
14.71 %
2.09 %
2.09 %
2.09 %
2.09 %
0.0 %
2.09 %
2.09 %
2.09 %
0.0 %
0.0 %
0.94 %
1.0 %
0.0 %
0.0 %
2.58 %
0.0 %
0.0 %
0.0 %
0.0 %
0.9 %
74.52 %
10.47 %
64.01 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
4.33 %
0.0 %
0.0 %
0.0 %
0.0 %
1.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
100.0 %
0.0 %
99.19 %
99.19 %
99.19 %
99.19 %
99.19 %
99.19 %
99.19 %
99.19 %
99.19 %
99.19 %
99.19 %
99.36 %
99.19 %
99.19 %
0.0 %
0.0 %
97.08 %
97.08 %
97.08 %
97.08 

***Deal with the data type warning***

Some rows in the csv have column shifted values ie value for column 1 is in column 2, value for column 2 in column 3 ...
There are very few so they will be added to a seperate list

Find how many different data types are in column 19

In [12]:
column_types = sample_accepted_df.iloc[:, 19].apply(type).unique()
print(column_types.size)
display(column_types)

1


array([<class 'float'>], dtype=object)

There are 2 different datatypes in the column.

Find the different values

In [5]:
sample_accepted_df.iloc[:,19].unique()

array([nan,
       'We knew that using our credit cards to finance an adoption would squeeze us, but then medical and other unexpected expenses made the situation almost impossible. We are a stable family in a stable community. We just need to break a cycle of debt that is getting worse.'],
      dtype=object)

In [6]:
row = sample_accepted_df[pd.notna(sample_accepted_df.iloc[:, 19])]
display(row)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
1480,361774,NaN,6400.0,6400.0,6400.0,36 months,8.49,202.01,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


Drop the problem row

In [11]:
sample_accepted_df.drop(index = 1480, inplace = True)

Instead of 200000 now there are 19,999 rows. The problem Row has been dropped. 

#### Check all columns

In [26]:
for column in sample_accepted_df.columns:
    # get the unique types for each column
    type_array = sample_accepted_df[column].apply(type).unique()
    
    # if there are more than 1 type in a column
    if type_array.size > 1:
        print(sample_accepted_df[column].dtypes)
        #filter out columns with NAN values
        print(pd.notna(sample_accepted_df[column].unique()))
        #calculate the unique values
        
        #row = sample_accepted_df[pd.notna(sample_accepted_df.iloc[:, 19])]
        
        print('Column name: ',column)
        print(type_array)
        
    

object
[ True  True  True ...  True  True  True]
Column name:  emp_title
[<class 'str'> <class 'float'>]
object
[ True  True  True  True  True  True  True  True  True  True  True False]
Column name:  emp_length
[<class 'str'> <class 'float'>]
object
[False  True]
Column name:  desc
[<class 'float'> <class 'str'>]
object
[ True  True False  True  True  True  True  True  True  True  True  True
  True]
Column name:  title
[<class 'str'> <class 'float'>]
object
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False  True]
Column name:  last_pymnt_d
[<class 'str'> <class 'float'>]
object
[False  True  True]
Column name:  next_pymnt_d
[<class 'float'> <class 'str'>]
object
[False  True]
Column name:  verification_status_joint
[<class 'float'> <class 'str'>]
object
[False  True]
Column name:  hardship_type


In [8]:
sample_accepted_df.index

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       19990, 19991, 19992, 19993, 19994, 19995, 19996, 19997, 19998, 19999],
      dtype='int64', length=19999)

In [9]:
sample_accepted_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19999 entries, 0 to 19999
Columns: 151 entries, id to settlement_term
dtypes: float64(114), int64(1), object(36)
memory usage: 23.2+ MB


The are too many columns to analyze at once

#### Seperate Columns by datatype

In [10]:
numeric_sample_accepted_df = sample_accepted_df.select_dtypes(include=['object'])
numeric_sample_accepted_df

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,...,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
0,36 months,C,C4,leadman,10+ years,MORTGAGE,Not Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1,36 months,C,C1,Engineer,10+ years,MORTGAGE,Not Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
2,60 months,B,B4,truck driver,10+ years,MORTGAGE,Not Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
3,60 months,C,C5,Information Systems Officer,10+ years,MORTGAGE,Source Verified,Dec-2015,Current,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
4,60 months,F,F1,Contract Specialist,3 years,MORTGAGE,Source Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,60 months,C,C1,Civil Aviation Security Specialist,10+ years,RENT,Not Verified,Dec-2015,Current,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
19996,60 months,C,C4,Sales manager,10+ years,OWN,Source Verified,Dec-2015,Charged Off,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
19997,36 months,B,B4,Teacher,8 years,RENT,Not Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
19998,36 months,B,B4,Business Banking Specialist,3 years,MORTGAGE,Source Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN


In [27]:
empty_columns = numeric_sample_accepted_df.columns[numeric_sample_accepted_df.isna().all()]
empty_columns

NameError: name 'numeric_sample_accepted_df' is not defined

In [12]:
rows, cols = sample_accepted_df.shape
print('Number of Rows: ',rows)
print('Number of Columns: ', cols)

Number of Rows:  19999
Number of Columns:  151


In [13]:
sample_accepted_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19999 entries, 0 to 19999
Columns: 151 entries, id to settlement_term
dtypes: float64(114), int64(1), object(36)
memory usage: 23.2+ MB


In [15]:
sample_accepted_df.isnull().sum()

id                           0
member_id                19999
loan_amnt                    0
funded_amnt                  0
funded_amnt_inv              0
                         ...  
settlement_status        19416
settlement_date          19416
settlement_amount        19416
settlement_percentage    19416
settlement_term          19416
Length: 151, dtype: int64

sam

In [16]:
objectsample_accepted_df.select_dtypes(object)

NameError: name 'objectsample_accepted_df' is not defined

            # if mixed column datatypes are found raise exception
            #HAVE TO GET DF DOWN TO SIZE WHERE IF I CAN CONCAT ALL TOGETHER AND EXPORT A NICE CLEAN CSV. DO ON PC IF NEEDED
            #OPTIMIZE ISH THE DATA TRAIN SPLIT WHEN DOING ML TO WINDOW DRESS

        #from pympler import asizeof
        #sampled_dataframes_size = asizeof.asizeof(sampled_dataframes)
        #print(sampled_dataframes_size)
        #print(sampled_dataframes_size/1000000, 'mb')

        #pathlib is used to ensure compatibility across operating systems
#if you have sufficient ram you can uncomment the other data destination and analyze the full dataset
try:
    data_destination = Path('../Data/Lending_club/accepted_2007_to_2018Q4.csv')
    #data_destination = Path('../Data/Lending_club/sample_accepted_2007_to_2018Q4.csv', low_memory = False)
    df= pd.read_csv(data_destination, low_memory=False)
    df.head(10)
    
except FileNotFoundError as e:
    print(e.args[1])
    print('Check file name and location')

## Conclusion

- sample_accepted_df.values
- 

### Resources used:

- https://stackoverflow.com/questions/3777301/how-to-call-a-shell-script-from-python-code
- https://stackoverflow.com/questions/51325601/how-to-stop-my-pandas-data-table-from-being-truncated-when-printed